## MSTICPy and Notebooks in InfoSec
---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Acquiring Data Using MSTICPy</a>

---

## What this session covers:
 - Setting up query providers
 - Connecting to providers
 - Querying for data
 - Offline data options

## Prerequisites
- Python >= 3.8 Environment
- Jupyter installed
- MSTICPy
- The msticpyconfig.yaml file you recently populated


### MSTICPy has a number of supported providers
- Microsoft Sentinel
- Microsoft Defender/Defender for Endpoint
- Splunk
- Sumologic
- Microsoft Graph
- Local data
- Mordor/Security Datasets
- Kusto/Azure Data Explorer
- Azure Resource Graph

These provide way to connect to and query data from these sources in a structured and standardized way.<br>
The providers also provide a way to create, store and call templated queries simply and easily.

Ref: https://msticpy.readthedocs.io/en/latest/DataAcquisition.html

In [ ]:
#Set up MSTICPy
%env MSTICPYCONFIG=./msticpyconfig.yaml
import msticpy as mp 
mp.init_notebook()

The QueryProvider handles this functionality and can be configured to work with the supported data sources.

`list_data_environments` shows us the names of the providers avalibale to us.

In [ ]:
mp.QueryProvider.list_data_environments()

You can then pass the name of the required provider to `QueryProvider`.

In [ ]:
qry_prov = mp.QueryProvider("MSSentinel")

Once created the next step is to connect the provider to the source and authenticate. <br>
When connecting we need to pass in the connection string of the source to connect to, this can either be as paratmers to the connect function, or from our configuration file.<br>

The authentication method for the provider will depend on the type of providers, and what is supported.

Below we will connect with a specific connection string, and the default auth method for this provider - Device Code.

In [ ]:
la_connection_string = f'loganalytics://code().tenant("72f988bf-86f1-41af-91ab-2d7cd011db47").workspace("8ecf8077-cf51-4820-aadd-14040956f35d")'
qry_prov.connect(connection_str=la_connection_string)

To use the a settings from our config instead of the connection string we can use WorkspaceConfig to collect those from file and pass them to the connection method.<br>
Whilst the default for many providers is Device Code authenication, other methods such as login via the Azure CLI can be easeir to use, so here we are going to use that. The advantage of this is that you don't need to authenticate every time as the Azure CLI credentials are cached on your machine.

In [ ]:
!az login

In [ ]:
qry_prov = mp.QueryProvider("MSSentinel")
qry_prov.connect(mp.WorkspaceConfig(), mp_az_auth=['cli'])

Once connected we can start running queries to get data.
We can do this with the built in queries or with our own queries.

We will start with the built in queries. We can list the avaliable queries with `list_queries`.

Ref: https://msticpy.readthedocs.io/en/latest/DataAcquisition.html#built-in-data-queries

In [ ]:
qry_prov.list_queries()

We can also use `browse` to get a clearer view of whats avaliable

In [ ]:
qry_prov.browse()

In [ ]:
qry_prov.Azure.list_all_signins_geo()

In [ ]:
df = qry_prov.Azure.list_all_signins_geo()
df.head()

Some queries require parameters such as a account or host name to search for results in.

In [ ]:
office_activity = qry_prov.Office365.list_activity_for_account(account_name="KDickens@seccxp.ninja")
office_activity.head()

We can customize built in queries with by adding query items to them.

In [ ]:
office_activity_filtered = qry_prov.Office365.list_activity_for_account(
    account_name="KDickens@seccxp.ninja",
    add_query_items="| where Operation != 'MailItemsAccessed'"
)
office_activity_filtered.head()

You can also add your own built in queries by specifying them in a yaml file and adding the required path to your msticpyconfig.yaml file. 

We can also use `exec_query` to run our own queries.

In [ ]:
query = "OfficeActivity | where TimeGenerated > ago(7d) | where UserId =~ 'KDickens@seccxp.ninja' | summarize count() by Operation"
custom_query_df = qry_prov.exec_query(query)
custom_query_df

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">1st Exercise - Run a query</a>

Execute a query against the created qry_prov. This can be a built in query or a custom query - its up to you.

<details>
<summary>Hints...</summary>
<ul>
<li>If you add "print" as a parameter when calling a query it will print out the query rather than executing it.</li>
</ul>
</details>


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Kusto</a>

---

Sentinel isn't the only data provider avaliable and we have plenty more that we can use to connect to.<br>
Kusto is a popular data source for a lot of uses.

Before we can use Kusto we need to set add some more config items to our msticpyconfig.yaml file. In this case we are going to add our Kusto cluster at `https://msticpytraining.eastus.kusto.windows.net`.

In [ ]:
from msticpy.config import  MpConfigEdit

MpConfigEdit()



## <a style="border: solid; padding:5pt; color:black; background-color:#309030">2nd Exercise - Kusto</a>

1 . Connect to the kusto cluster https://msticpytraining.eastus.kusto.windows.net/ and the msticpydata database. <br>
2. Run a query to understand the schema of the Syslog table


<details>
<summary>Hints...</summary>
<ul>
<li>The Kusto data provider is called simply "Kusto"</li>
<li>The`cluster` and `database` parameters are key here.</li>
<li>https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProv-Kusto.html has the details you need</li>
</ul>
</details>


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Microsoft Defender</a>

---
Some data providers have different connection options, for example the Microsoft Defender for Endpoint and Microsoft 365 Defender APIs require a client applicaiton to handle authentication.<br>
You can pass in these application details when connecting but if we are using an applicaiton secret its better to keep these in KeyVault and refence them in our config file.

In [ ]:
import msticpy as mp 

mp.init_notebook()

You can store multiple instances in your config file. To select what instance to connect to use the `instance` keyword.<br>
In this example we will connect to our pre-configured Training insance.

Ref: https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProv-MSDefender.html#connecting-to-m365-defender

In [ ]:
defender_prov = mp.QueryProvider("M365D")
defender_prov.connect(instance="Training")

In [ ]:
defender_prov.browse()

We can also execute our own queries in the same format as with the other providers.

In [ ]:
defender_prov.exec_query("DeviceInfo | take 10")

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">3rd Exercise - Defender Investigation</a>

1. Find the IP address associated with alerts relating to the url 'davlenwindows.com' on 10/14/2022
2. Find all the hosts that have connected to that IP address since 10/14/2022
3. Get the file hash of the initiating process for these connections and get all the files names assocaited with this hash


<details>
<summary>Hints...</summary>
<ul>
<li>You can do this with built in queries or your own queries</li>
<li>The Query Browser is your friend `qry_prov.browse()`</li>
<li>Don't forget you can use add_query_items to add to the built in queries to customize the returned data.</li>
</ul>
</details>

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Azure Resource Graph</a>

---


The Azure Resource graph provides a way to get details about Azure Resources using KQL, this is something that is really useful to adding context during an investigation.<br>
Below we are going to load our Resource Graph provider and connect using the Azure CLI tokens that we generated earlier.

In [ ]:
res_qry_prov = mp.QueryProvider("ResourceGraph")
res_qry_prov.connect(auth_methods=["cli"])

As with the other providers we can use in built queries or write our own custom queries. Hopefully by now you are familiar with this model and concept.


## <a style="border: solid; padding:5pt; color:black; background-color:#309030">4th Exercise - Azure Resource Graph</a>

 1 . Find out how many KeyVaults that you have access to. <br>
 2. What resources exist in the msticpy resource group.<br>
 3. Find the Key Vault that is detailed in your msticpyconfig.yaml file<br>


<details>
<summary>Hints...</summary>
<ul>
<li>https://learn.microsoft.com/en-us/azure/governance/resource-graph/samples/starter?tabs=azure-cli gives you some query examples</li>
<li>microsoft.keyvault/vaults</li>
<li>You will need to use .exec_query here</li>
</ul>
</details>



## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Bonus Exercise - Azure Resource Graph</a>

CDOC recived a report that the VM MSTIC-DSVM has been compromised. You need to answer the following questions:
1. Is this a real host?
2. Is it currently in use?
3. What IPs is it assocaited with?
4. Is it a production host?
5. What other resources might have been compromised?
6. Are there any users we can contact about this host?
